# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from utils import stratified_train_test_group_kfold
from utils import model_test_classification

from classification_codes import knc_gridsearch



0.10.1


# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


In [4]:
#! mkdir -p 'Model/Class/TrainedImageNet/Up/'

# 3. Runing Cross-Validation

In [6]:
N_neighbors = [i for i in range(3, 15)]
Weights = ['uniform', 'distance']
Algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
P = [i for i in range(1, 6)]


def cross_10_folds_knn(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug3/KNC_{fold}_model.sav'
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = knc_gridsearch(train_X, train_Y, val_X, val_Y, N_neighbors, Weights, Algorithm, P, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_knn(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train)}, std: {np.std(best_train)}, val_acc_mean: {np.mean(best_val)}, std: {np.std(best_val)}')

<class 'generator'>
best_train_acc: 0.851, best_val_acc: 0.667
best_train_acc: 0.867, best_val_acc: 0.708
fold: 0 done!
<class 'generator'>
best_train_acc: 0.849, best_val_acc: 0.750
best_train_acc: 0.855, best_val_acc: 0.792
best_train_acc: 0.862, best_val_acc: 0.833
best_train_acc: 0.777, best_val_acc: 0.875
best_train_acc: 1.000, best_val_acc: 0.917
fold: 1 done!
<class 'generator'>
best_train_acc: 0.855, best_val_acc: 0.500
best_train_acc: 1.000, best_val_acc: 0.542
best_train_acc: 1.000, best_val_acc: 0.583
best_train_acc: 0.783, best_val_acc: 0.625
best_train_acc: 0.730, best_val_acc: 0.708
fold: 2 done!
<class 'generator'>
best_train_acc: 0.848, best_val_acc: 0.583
best_train_acc: 0.854, best_val_acc: 0.667
fold: 3 done!
<class 'generator'>
best_train_acc: 0.863, best_val_acc: 0.750
best_train_acc: 1.000, best_val_acc: 0.792
fold: 4 done!
<class 'generator'>
best_train_acc: 0.860, best_val_acc: 0.478
best_train_acc: 0.857, best_val_acc: 0.522
best_train_acc: 0.803, best_val_acc:

In [7]:
import numpy as np
from sklearn.metrics import f1_score

X, Y = train_val_X, train_val_Y
groups = train_val_groups
def f1score_fn(X, Y):

    F1_score_train = []
    F1_score_val = []
    for fold in range(10):
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=fold)
        #train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        model_path = f'classification/ImageNet/aug3/KNC_{fold}_model.sav'
        
        PATH = os.path.join('Models', model_path)
        
        loaded_model = pickle.load(open(PATH, 'rb'))

        pred_val_Y = loaded_model.predict(val_X)
        pred_train_Y = loaded_model.predict(train_X)

        f_score_train = f1_score(train_Y, pred_train_Y, average='macro')#'weighted')
        f_score_val = f1_score(val_Y, pred_val_Y, average='macro')#'weighted')

        F1_score_train.append(f_score_train)
        F1_score_val.append(f_score_val)
    
    return F1_score_train, F1_score_val



F1_score_train, F1_score_val = f1score_fn(X, Y)


print(f'f1_train: {np.mean(F1_score_train)}, std: {np.std(F1_score_train)}, f1_val: {np.mean(F1_score_val)}, std: {np.std(F1_score_val)}')


<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
f1_train: 0.7990899606851487, std: 0.1733597585995738, f1_val: 0.6300325205372991, std: 0.12406084090057731


10
[[1.8, 1.1, 0.1], [3.4, 7.7, 1.9], [0.8, 2.3, 7.9]]


# 4. Model Testing

In [4]:
import numpy as np
from sklearn.metrics import f1_score




best_fold =1

model_path = f'classification/ImageNet/aug3/KNC_{_best_fold}_model.sav'


# Fitting different train in the same hyperparameters
best_test, confusion_matrix_test = model_test_classification(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)



print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
[0.6296296296296297, 0.7037037037037037, 0.6296296296296297, 0.6296296296296297, 0.7037037037037037, 0.6666666666666666, 0.5185185185185185, 0.6296296296296297, 0.6296296296296297, 0.6296296296296297]
test_acc_mean: 0.637, std: 0.049, f1_train: 1.0, std: 0.0, f1_val: 0.5438593776364675, std: 0.15924069064776183
5th confusion matrix:  [[2, 1, 0], [4, 9, 0], [0, 3, 8]]


In [8]:
best_fold =1

model_path = f'classification/ImageNet/aug3/KNC_{best_fold}_model.sav'

PATH = os.path.join('Models', model_path)

#filename = f'Model/Class/TrainedImageNet/Up/KNN_{best_fold}_model.sav'
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'distance'}

In [ ]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print([list(item) for item in np.mean(cm_list, axis=0)])